In [1]:
!pip install transformers
!pip install torch
!pip install nltk

In [2]:
!pip install fastai

In [3]:
import pandas as pd
from fastai.text.all import *
import string
from nltk.corpus import stopwords

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df = pd.read_csv("/kaggle/input/quotes-500k/quotes.csv")
df.head()

,quote,author,category
0,"I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.",Marilyn Monroe,"attributed-no-source, best, life, love, mistakes, out-of-control, truth, worst"
1,"You've gotta dance like there's nobody watching,Love like you'll never be hurt,Sing like there's nobody listening,And live like it's heaven on earth.",William W. Purkey,"dance, heaven, hurt, inspirational, life, love, sing"
2,You know you're in love when you can't fall asleep because reality is finally better than your dreams.,Dr. Seuss,"attributed-no-source, dreams, love, reality, sleep"
3,A friend is someone who knows all about you and still loves you.,Elbert Hubbard,"friend, friendship, knowledge, love"
4,Darkness cannot drive out darkness: only light can do that. Hate cannot drive out hate: only love can do that.,"Martin Luther King Jr., A Testament of Hope: The Essential Writings and Speeches","darkness, drive-out, hate, inspirational, light, love, peace"


In [6]:
df=df.drop("author",axis=1)

In [7]:
df.head()

,quote,category
0,"I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.","attributed-no-source, best, life, love, mistakes, out-of-control, truth, worst"
1,"You've gotta dance like there's nobody watching,Love like you'll never be hurt,Sing like there's nobody listening,And live like it's heaven on earth.","dance, heaven, hurt, inspirational, life, love, sing"
2,You know you're in love when you can't fall asleep because reality is finally better than your dreams.,"attributed-no-source, dreams, love, reality, sleep"
3,A friend is someone who knows all about you and still loves you.,"friend, friendship, knowledge, love"
4,Darkness cannot drive out darkness: only light can do that. Hate cannot drive out hate: only love can do that.,"darkness, drive-out, hate, inspirational, light, love, peace"


In [8]:
df = df[["quote", "category"]].dropna()

In [9]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation and special characters
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Remove stop words
    words = text.split()
    words = [word for word in words if word not in stop_words]
    text = " ".join(words)
    
    return text

In [10]:
df["quote"] = df["quote"].apply(preprocess_text)

In [11]:
dialogues = []
for _, row in df.iterrows():
    quote = row["quote"]
    category = row["category"]
    dialogue = f"User: Tell me a quote about {category}.\nBot: {quote}"
    dialogues.append(dialogue)

dialogue_df = pd.DataFrame({"dialogue": dialogues})

In [12]:
train_ratio = 0.9
train_size = int(len(dialogue_df) * train_ratio)
train_df = dialogue_df[:train_size]
valid_df = dialogue_df[train_size:]

train_df.to_csv("train.csv", index=False)
valid_df.to_csv("valid.csv", index=False)

In [13]:
# Fine-tune the Language Model
data_lm = TextDataLoaders.from_csv(".", "train.csv", valid="valid.csv", text_col="dialogue", is_lm=True, bs=128, tokenizer=Tokenizer.from_df("quote"))

learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3).to_fp16()
learn.fine_tune(6, 5e-2)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

epoch,train_loss,valid_loss,time
0,3.972884,3.866833,28:12


epoch,train_loss,valid_loss,time
0,3.560128,3.569770,29:52
1,3.538095,3.517415,29:39
2,3.429698,3.464366,29:51
3,3.307730,3.422264,29:50
4,3.213117,3.405540,29:57
5,3.048753,3.433825,30:41


In [14]:
def postprocess_response(response):
    response = response.strip()
    response = response.replace("User:", "").strip()
    response = response.capitalize()
    if not response.endswith("."):
        response += "."
    
    return response

In [ ]:
greeting = "Bot: Hi! I'm the Quote Bot. Ask me for quotes about different topics!"

print(greeting)
while True:
    user_input = input("User: ")

    # Exit the loop if the user enters "exit"
    if user_input.lower() == "exit":
        print("Bot: Goodbye!")
        break

    response = learn.predict(user_input, n_words=50, temperature=0.3)
    postprocessed_response = postprocess_response(response)
    print("Bot:", postprocessed_response)